This notebook demonstrates the inference of two text detoxification models in https://github.com/s-nlp/detox.

It mostly copies these two files, with adjustments for the Colab environment:
-  https://github.com/s-nlp/detox/blob/main/emnlp2021/style_transfer/condBERT/condbert_inference.ipynb
- https://github.com/s-nlp/detox/blob/main/emnlp2021/style_transfer/paraGeDi/gedi_inference.ipynb

# Getting the code and installing the requirements

In [1]:
!git clone https://github.com/s-nlp/detox

Cloning into 'detox'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 186 (delta 54), reused 118 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (186/186), 17.92 MiB | 22.85 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [2]:
! pip install -r detox/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.4/677.4 kB 29.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

# ParaGeDi

## Setting up

In [ ]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [ ]:
# adding the path to the condebert code root
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/content/detox/emnlp2021/style_transfer/condBERT


In [ ]:
from importlib import reload

In [ ]:
import condbert
reload(condbert)
from condbert import CondBertRewriter

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

In [ ]:
device = torch.device('cpu')  # please change it to e.g. 'cuda:0' if you want to use a GPU

In [ ]:
vocab_root = 'detox/emnlp2021/style_transfer/condBERT/vocab/'
toxic_data_path = 'detox/emnlp2021/data/test/test_10k_toxic'

In [10]:
import torch
import numpy as np

In [11]:
add_sys_path('detox/emnlp2021/style_transfer/paraGeDi')

/content/detox/emnlp2021/style_transfer/paraGeDi


In [12]:
from importlib import reload
import gedi_adapter
reload(gedi_adapter)
from gedi_adapter import GediAdapter

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
t5name = 's-nlp/t5-paraphrase-paws-msrp-opinosis-paranmt'
model_path = 's-nlp/gpt2-base-gedi-detoxification'
clf_name = 's-nlp/roberta_toxicity_classifier_v1'

In [14]:
import text_processing
reload(text_processing);

In [15]:
import torch
#device = torch.device('cuda:0')
device = torch.device('cpu')

## Loading the model

In [16]:
tokenizer = AutoTokenizer.from_pretrained(t5name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [17]:
para = AutoModelForSeq2SeqLM.from_pretrained(t5name)
para.resize_token_embeddings(len(tokenizer))

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Embedding(32100, 768)

In [18]:
para.to(device);
para.eval();

In [19]:
gedi_dis = AutoModelForCausalLM.from_pretrained(model_path)

config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

Some weights of the model checkpoint at s-nlp/gpt2-base-gedi-detoxification were not used when initializing GPT2LMHeadModel: ['logit_scale', 'bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
NEW_POS = tokenizer.encode('normal', add_special_tokens=False)[0]
NEW_NEG = tokenizer.encode('toxic', add_special_tokens=False)[0]

In [21]:
# add gedi-specific parameters
if os.path.exists(model_path):
    w = torch.load(model_path + '/pytorch_model.bin', map_location='cpu')
    gedi_dis.bias = w['bias']
    gedi_dis.logit_scale = w['logit_scale']
    del w
else:
    gedi_dis.bias = torch.tensor([[ 0.08441592, -0.08441573]])
    gedi_dis.logit_scale = torch.tensor([[1.2701858]])
print(gedi_dis.bias, gedi_dis.logit_scale)

tensor([[ 0.0844, -0.0844]]) tensor([[1.2702]])


## Inference example

The cell below uses random sampling, so it's ok if it doesn't reproduce.

In [22]:
%%time

dadapter = GediAdapter(model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=5, target=1, neg_code=NEW_NEG, pos_code=NEW_POS, lb=None, ub=None)
text = 'The internal policy of Trump is flawed.'
print('====BEFORE====')
print(text)
print('====AFTER=====')
inputs = tokenizer.encode(text, return_tensors='pt').to(para.device)
result = dadapter.generate(inputs, do_sample=True, num_return_sequences=3, temperature=1.0, repetition_penalty=3.0, num_beams=1)
for r in result:
    print(tokenizer.decode(r, skip_special_tokens=True))

====BEFORE====
The internal policy of Trump is flawed.
====AFTER=====


/usr/local/lib/python3.11/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


the regime of trump is corrupt.
his internal behavior today is deficient!
his domestic policy is absolutely corrupt.
CPU times: user 5.86 s, sys: 2.59 ms, total: 5.86 s
Wall time: 6.13 s


In [23]:
import gc

def cleanup():
    gc.collect()
    if torch.cuda.is_available() and device.type != 'cpu':
        with torch.cuda.device(device):
            torch.cuda.empty_cache()

In [24]:
gedi_dis.to(device);
gedi_dis.bias = gedi_dis.bias.to(device)
gedi_dis.logit_scale = gedi_dis.logit_scale.to(device)
gedi_dis.eval();

In [25]:
with open(toxic_data_path, 'r') as f:
    test_data = [line.strip() for line in f.readlines()]
print(len(test_data))

10000


In [26]:
dadapter = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=10, target=0, neg_code=NEW_NEG, pos_code=NEW_POS,
    reg_alpha=3e-5, ub=0.01
)

In [27]:
def paraphrase(text, n=None, max_length='auto', beams=2):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(dadapter.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    result = dadapter.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False, temperature=0.0, repetition_penalty=3.0, max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    texts = [text_processing.text_postprocess(t) for t in texts]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [28]:
paraphrase(test_data[:3])

["You'd be a bad guy. Oh, yeah.",
 'As snooty and overbearing as his boss.',
 'A bad society does bad things, and votes for bad politicians.']

Expected output

```
["You'd be a bad guy. Oh, yeah.",
 'As snooty and overbearing as his boss.',
 'A bad society does bad things, and votes for bad politicians.']
 ```

## Inference with reranking by classifier

In [29]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

In [30]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [31]:
predict_toxicity(['hello world', 'hello aussie', 'hello fucking bitch'])

array([5.052513e-05, 8.788541e-05, 9.996809e-01], dtype=float32)

In [32]:
from gedi_adapter import GediAdapter


adapter2 = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer,
    gedi_logit_coef=10,
    target=0, pos_code=NEW_POS,
    neg_code=NEW_NEG,
    reg_alpha=3e-5,
    ub=0.01,
    untouchable_tokens=[0, 1],
)


def paraphrase(text, max_length='auto', beams=5, rerank=True):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(adapter2.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    attempts = beams
    out = adapter2.generate(
        inputs,
        num_beams=beams,
        num_return_sequences=attempts,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        output_scores=True,
        return_dict_in_generate=True,
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]

    if rerank:
        scores = predict_toxicity(results)

    results = [text_processing.text_postprocess(t) for t in results]
    out_texts = []
    for i in range(len(texts)):
        if rerank:
            idx = scores[(i*attempts):((i+1)*attempts)].argmin()
        else:
            idx = 0
        out_texts.append(results[i*attempts+idx])
    return out_texts

torch.manual_seed(0)
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=3)

["Fick 'Emmy.",
 "You'd be wrong!",
 "You'll remind me of chump?",
 'Must be a Terrorist!']

In [33]:
paraphrase(["I bet you only hire outsiders because of some stupid policy.", "I can't believe you people still have jobs. You are all morons!", "I refuse to take financial advice from a foreigners. Get me someone else.", "Typical immigrants behavior. Can I talk to someone competent?"], beams=3)

["I'd bet that you only hire outside contractors out of the rules.",
 "I can't even believe that you still have jobs.",
 "I've refused to take financial advice from foreigners.",
 'can I talk to someone competent?']

In [34]:
batch_size = 2

In [35]:
'''
import os
from tqdm.auto import tqdm, trange

cleanup()

lines = test_data[:10]


for i in trange(int(len(lines) / batch_size + 1)):
    if i % 10 == 0:
        cleanup()
    t = i * batch_size
    batch = [line.strip() for line in lines[t:(t+batch_size)]]
    if not batch:
        continue

    try:
        res = paraphrase(batch, max_length='auto', beams=10)
    except RuntimeError:
        print('runtime error for batch ', i)
        try:
            cleanup()
            res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]
        except RuntimeError:
            print('runtime error for batch ', i, 'even with batch size 1')
            res = batch
            cleanup()
    for out in res:
        print(out)
'''

"\nimport os\nfrom tqdm.auto import tqdm, trange\n\ncleanup()\n\nlines = test_data[:10]\n\n\nfor i in trange(int(len(lines) / batch_size + 1)):\n    if i % 10 == 0:\n        cleanup()\n    t = i * batch_size\n    batch = [line.strip() for line in lines[t:(t+batch_size)]]\n    if not batch:\n        continue\n\n    try:\n        res = paraphrase(batch, max_length='auto', beams=10)\n    except RuntimeError:\n        print('runtime error for batch ', i)\n        try:\n            cleanup()\n            res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]\n        except RuntimeError:\n            print('runtime error for batch ', i, 'even with batch size 1')\n            res = batch\n            cleanup()\n    for out in res:\n        print(out)\n"

Expected output (when running on CUDA; on CPU it migth slightly differ):
```
You'd be right. You'd be right. You'd
As snooty and overbearing as its
A bad society does the wrong things, and votes for the wrong politicians.
President, he's the man. He's the man.
Colberg's a bad guy, and I'm a TSA.
Dubious, dubious, dubious, dubious.
I like you. I think you'll be an intelligent man and your contributions will be a big deal.
Luna's oh-so-stunnel, immature girl......who has no idea how to do it, which smells bad, I'd like to see
Mormonis is the wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The right thing to do. The right thing to do. The right thing to do
You'd be a bad guy, uninitiated.
```